# 🎹 Constant-Q Transform (CQT) Analyzer

This notebook allows you to upload an audio file and visualize its musical structure using the **Constant-Q Transform (CQT)**.

The CQT is similar to a spectrogram but uses a logarithmically spaced frequency axis, which corresponds closely to musical notes.

In [ ]:
# Environment Setup
import sys
import subprocess

try:
    import google.colab
    IN_COLAB = True
    print("🌐 Running on Google Colab")
    
    # Install dependencies
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "gradio", "librosa", "soundfile", "matplotlib"], check=True)
    print("✅ Dependencies installed.")
except ImportError:
    IN_COLAB = False
    print("💻 Running locally")

In [ ]:
import gradio as gr
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import os

print("✅ Libraries imported.")

In [ ]:
def analyze_cqt(audio_path):
    """Computes and plots the Constant-Q Transform."""
    if audio_path is None:
        return None, "Please upload an audio file."
    
    try:
        # Load audio
        y, sr = librosa.load(audio_path)
        
        # Compute CQT
        C = librosa.cqt(y, sr=sr)
        C_db = librosa.amplitude_to_db(np.abs(C), ref=np.max)
        
        # Plot
        fig, ax = plt.subplots(figsize=(12, 6))
        img = librosa.display.specshow(C_db, sr=sr, x_axis='time', y_axis='cqt_note', ax=ax, cmap='magma')
        fig.colorbar(img, ax=ax, format='%+2.0f dB')
        ax.set_title(f"Constant-Q Transform (CQT)\n{os.path.basename(audio_path)}")
        plt.tight_layout()
        
        return fig, "✅ Analysis Complete"
        
    except Exception as e:
        return None, f"❌ Error: {str(e)}"

In [ ]:
# Build the Gradio App
with gr.Blocks(title="CQT Analyzer") as demo:
    gr.Markdown("# 🎹 CQT Analyzer")
    gr.Markdown("Upload an audio file to visualize its musical content.")
    
    with gr.Row():
        with gr.Column():
            audio_input = gr.Audio(label="Upload Audio", type="filepath")
            analyze_btn = gr.Button("Analyze", variant="primary")
            status_output = gr.Textbox(label="Status", interactive=False)
        
        with gr.Column():
            cqt_plot = gr.Plot(label="CQT Spectrogram")

    analyze_btn.click(
        fn=analyze_cqt,
        inputs=[audio_input],
        outputs=[cqt_plot, status_output]
    )

# Handle case where setup cell wasn't run (IN_COLAB undefined)
try:
    share_mode = IN_COLAB
except NameError:
    share_mode = False

demo.launch(share=share_mode)